In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('finalData.csv')

In [3]:
df.head()

,Restaurant Name,Area,Restaurant type,Ratings,Number of ratings,price,Address
0,Cricket Ground Cafe,Badda,Cafe,4.1,169,1,"171/3 Middle Badda, হাতির ঝিল লিঙ্ক রোড"
1,Rosemery Restaurant,Badda,Bangladeshi,3.9,369,1,Madani Ave
2,Sanxi Thai Chinese,Badda,Chinese,3.9,1102,2,"House#1/B, Road#13, South"
3,The Artichoke,Badda,Restaurant,4.4,178,1,"Arma Majeda Malik Tower , Holding No# 215, Bir..."
4,Cabana Restaurant and Party Center(Roof Top),Badda,Restaurant,4.0,143,2,"House 04, Road -10, DIT Project, Merul Badda D..."


In [4]:
#Assign weight based on ratings and price

In [5]:
v = df['Number of ratings']
R = df['Ratings']
C = df['Ratings'].mean()
m = df['Number of ratings'].quantile(0.60)
price = df['price']
df['weighted_average'] = (((R*v)+(C*m))/(v+m))
df['price_average'] = (3/df['price'])

In [6]:
df.head()

,Restaurant Name,Area,Restaurant type,Ratings,Number of ratings,price,Address,weighted_average,price_average
0,Cricket Ground Cafe,Badda,Cafe,4.1,169,1,"171/3 Middle Badda, হাতির ঝিল লিঙ্ক রোড",4.030678,3.0
1,Rosemery Restaurant,Badda,Bangladeshi,3.9,369,1,Madani Ave,3.967601,3.0
2,Sanxi Thai Chinese,Badda,Chinese,3.9,1102,2,"House#1/B, Road#13, South",3.938162,1.5
3,The Artichoke,Badda,Restaurant,4.4,178,1,"Arma Majeda Malik Tower , Holding No# 215, Bir...",4.101837,3.0
4,Cabana Restaurant and Party Center(Roof Top),Badda,Restaurant,4.0,143,2,"House 04, Road -10, DIT Project, Merul Badda D...",4.008444,1.5


In [7]:
#Assign score 90 precent weight and 10 percent price
df['score'] = df['weighted_average']*0.90 + df['price_average']*0.10

In [8]:
df.head(5)

,Restaurant Name,Area,Restaurant type,Ratings,Number of ratings,price,Address,weighted_average,price_average,score
0,Cricket Ground Cafe,Badda,Cafe,4.1,169,1,"171/3 Middle Badda, হাতির ঝিল লিঙ্ক রোড",4.030678,3.0,3.927610
1,Rosemery Restaurant,Badda,Bangladeshi,3.9,369,1,Madani Ave,3.967601,3.0,3.870841
2,Sanxi Thai Chinese,Badda,Chinese,3.9,1102,2,"House#1/B, Road#13, South",3.938162,1.5,3.694346
3,The Artichoke,Badda,Restaurant,4.4,178,1,"Arma Majeda Malik Tower , Holding No# 215, Bir...",4.101837,3.0,3.991654
4,Cabana Restaurant and Party Center(Roof Top),Badda,Restaurant,4.0,143,2,"House 04, Road -10, DIT Project, Merul Badda D...",4.008444,1.5,3.757599


In [9]:
#Content based recommendation:

In [10]:
from sklearn.preprocessing import LabelEncoder
le_area = LabelEncoder()
le_type = LabelEncoder()
df['area_value'] = le_area.fit_transform(df['Area'])
df['type_value'] = le_type.fit_transform(df['Restaurant type'])

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['area_value'] = scaler.fit_transform(df[['area_value']])
df['type_value'] = scaler.fit_transform(df[['type_value']])
df['Ratings_value'] = scaler.fit_transform(df[['Ratings']])
df['Number of ratings value'] = scaler.fit_transform(df[['Number of ratings']])
df['price_value'] = scaler.fit_transform(df[['price']])


In [12]:
X = df[['Ratings_value','price_value','area_value','type_value','Number of ratings value']]
X

,Ratings_value,price_value,area_value,type_value,Number of ratings value
0,0.578947,0.0,0.0,0.229167,0.006217
1,0.473684,0.0,0.0,0.083333,0.016667
2,0.473684,0.5,0.0,0.312500,0.054963
3,0.736842,0.0,0.0,0.833333,0.006688
4,0.526316,0.5,0.0,0.833333,0.004859
...,...,...,...,...,...
993,0.526316,0.0,1.0,0.833333,0.006792
994,0.526316,0.5,1.0,0.833333,0.019122
995,0.263158,0.0,1.0,0.229167,0.004963
996,0.631579,0.5,1.0,0.979167,0.057367


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
newArray = cosine_similarity(X,X)

In [14]:
indices = pd.Series(df.index, index=df['Restaurant Name']).drop_duplicates()

In [15]:
def get_recommendations(title, cosine_sim=newArray):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
        
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df[['Restaurant Name','Area','Restaurant type','price','Ratings']].iloc[movie_indices]

In [28]:
def searchRestaurant(area, rest_type, price):
    newDf = df[(df['Area'] == area)&(df['Restaurant type'] == rest_type)&(df['price'] == int(price))]
    newDf = newDf.sort_values('score', ascending= False)
    return newDf[['Restaurant Name','Area','Restaurant type','price','Ratings','score']]

In [29]:
newDf.head(10)

,Restaurant Name,Area,Restaurant type,Ratings,Number of ratings,price,Address,weighted_average,price_average,score,area_value,type_value,Ratings_value,Number of ratings value,price_value
757,UniCafe Restaurant & Fun,Old Dhaka,Restaurant,4.1,599,2,36 Rankin St,4.055935,1.5,3.800342,0.652174,0.833333,0.578947,0.028683,0.5
748,Seagull Restaurant,Old Dhaka,Restaurant,4.1,585,2,"12th Floor, Hossain Tower, 116 Culvert Road",4.055406,1.5,3.799866,0.652174,0.833333,0.578947,0.027952,0.5
766,Spice & Herbs Restaurant,Old Dhaka,Restaurant,4.1,163,2,"90/91 SH Plaza, 1st Floor, Nazimuddin Road",4.030119,1.5,3.777107,0.652174,0.833333,0.578947,0.005904,0.5
761,Grand Darbar Restaurant,Old Dhaka,Restaurant,4.0,596,2,26 Hatkhola Rd,4.005194,1.5,3.754675,0.652174,0.833333,0.526316,0.028527,0.5
738,Royal Castle,Old Dhaka,Restaurant,4.0,1070,2,50 Lalbagh Rd,4.003703,1.5,3.753333,0.652174,0.833333,0.526316,0.053292,0.5
765,Bhooter Bari Restaurant,Old Dhaka,Restaurant,4.0,1824,2,"32/34/Ka, Lalbagh Road, (Oposite",4.002542,1.5,3.752288,0.652174,0.833333,0.526316,0.092685,0.5
781,Cafe M'eating,Old Dhaka,Restaurant,3.8,171,2,16/10 Rankin St,3.962663,1.5,3.716396,0.652174,0.833333,0.421053,0.006322,0.5
767,Dawat-E Kabab,Old Dhaka,Restaurant,3.8,281,2,"157, ডিষ্টিলারি রোড, Gandaria",3.941910,1.5,3.697719,0.652174,0.833333,0.421053,0.012069,0.5
752,Taj-E-Nawab,Old Dhaka,Restaurant,3.8,638,2,"Tower (4th Floor, AK Famous, 41 Rankin St",3.900357,1.5,3.660321,0.652174,0.833333,0.421053,0.030721,0.5
780,Grand Haveli,Old Dhaka,Restaurant,3.8,859,2,"3rd Floor, Rose Valley 29, Rankin Street",3.884957,1.5,3.646461,0.652174,0.833333,0.421053,0.042268,0.5


,Restaurant Name,Area,Restaurant type,price,Ratings
814,Dhaka Hotel and Restaurant,Ramna,Restaurant,2,4.1
766,Spice & Herbs Restaurant,Old Dhaka,Restaurant,2,4.1
748,Seagull Restaurant,Old Dhaka,Restaurant,2,4.1
757,UniCafe Restaurant & Fun,Old Dhaka,Restaurant,2,4.1
761,Grand Darbar Restaurant,Old Dhaka,Restaurant,2,4.0
823,Pizza Inn,Ramna,Pizza,2,4.1
738,Royal Castle,Old Dhaka,Restaurant,2,4.0
734,Radhuni Restaurant,Naya Paltan,Restaurant,2,4.0
742,Dosa Express,Old Dhaka,SouthIndian,2,4.1
813,Da Rooftop,Ramna,Restaurant,2,4.0


In [47]:
area = input("enter restaurant area: ")
restType = input("enter restaurant type: ")
price = input("enter restaurant price: ")
searchRestaurant(area,restType,price)

enter restaurant area: Banani
enter restaurant type: Fast Food
enter restaurant price: 2


,Restaurant Name,Area,Restaurant type,price,Ratings,score
96,Takeout Banani,Banani,Fast Food,2,4.4,4.074954
49,"Madchef Banani, Dhaka.",Banani,Fast Food,2,4.3,3.974545
48,Herfy Banani,Banani,Fast Food,2,4.2,3.888131
86,BFC,Banani,Fast Food,2,4.1,3.821443
50,Urban Spoon,Banani,Fast Food,2,4.1,3.798477
25,KFC,Banani,Fast Food,2,4.0,3.753002


In [49]:
get_recommendations("BFC")

,Restaurant Name,Area,Restaurant type,price,Ratings
48,Herfy Banani,Banani,Fast Food,2,4.2
25,KFC,Banani,Fast Food,2,4.0
50,Urban Spoon,Banani,Fast Food,2,4.1
49,"Madchef Banani, Dhaka.",Banani,Fast Food,2,4.3
141,KFC,Banasree,Fast Food,2,3.9
88,Sajna Restaurant,Banani,Indian,2,4.1
110,New Cathay,Banani,Chinese,2,3.9
2,Sanxi Thai Chinese,Badda,Chinese,2,3.9
39,"Cheong Shing Restaurant, Dhaka.",Banani,Chinese,2,4.2
70,KFD Express,Banani,Chinese,2,4.3
